# Drift detection

In the context of machine learning, data drift refers to a change in the statistical properties of the input data over time compared to the data on which the model was originally trained. This can result in the model's performance deteriorating, as it may no longer generalise well to new, unseen data distributions.

There are several types of data drift, including:

- **Covariate Drift (Feature Drift)**: The distribution of the input features changes over time, but the relationship between features and target remains the same. For example, if a model was trained on data where a certain feature had a specific range of values, and over time that feature's values shift significantly, this can lead to covariate drift.

- **Prior Probability Drift (Label Drift)**: The distribution of the target variable changes. For instance, if a model was trained to predict energy consumption patterns during a specific season, and the seasonal patterns change due to external factors.

- **Concept Drift**: The relationship between the input features and the target variable changes. For example, if a model was trained to predict energy consumption based on weather data, and over time, the relationship between weather patterns and energy consumption changes due to new technologies or behavioural changes.

Detecting and addressing data drift is crucial for maintaining the performance of machine learning models in production. Techniques for handling data drift include:

- Monitoring input data in the prediction phase.

- Monitoring model performance (accuracy, precision, recall, etc.) over time.
  
- Periodically retraining the model with new data to adapt to changes.

Skforecast provides the class `RangeDriftDetector` to detect covariate drift in both single and multiple time series, as well as in exogenous variables. It checks whether the input data (lags and exogenous variables) used for predict new values fall within the range of the data used to train the model. Its API follows the same design as the forecasters: the data used to train a forecaster can also be used to fit the `RangeDriftDetector`, and the data passed for prediction can be used to check for drift.

<div role="note"
    style="background: rgba(0,184,212,.08); border-left: 6px solid #00b8d4;
          border-radius: 6px; padding: 10px 12px; margin: 1em 0;">

<p style="display:flex; align-items:center; font-size:1rem; color:#00b8d4;
          margin:0 0 6px 0; font-weight:600;">
  <span style="margin-right:6px; font-size:1.125em;">✏️</span>
  <strong style="font-size:18px;">Note</strong>
</p>

<p style="margin:0; color:inherit;">
  This module is in active development, we expect to add more features and improvements in future releases.
</p>

</div>

## Libraries and data

The dataset used in this user guide consists of information on the number of users of a bicycle rental service, in addition to weather variables and holiday data. Two of the variables in the dataset, `holiday` and `weather`, are categorical.

In [24]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
from skforecast.drift_detection import RangeDriftDetector

In [25]:
# Simulated data
# ==============================================================================
rgn = np.random.default_rng(123)
series = pd.Series(
    rgn.normal(loc=10, scale=2, size=100),
    index=pd.date_range(start="2020-01-01", periods=100),
    name="y",
)
exog = pd.DataFrame({
        'exog_1': rgn.normal(loc=10, scale=2, size=100),
        'exog_2': rgn.choice(['A', 'B', 'C', 'D', 'E'], size=100)
    }, index=series.index
)
display(series.head())
display(exog.head())

2020-01-01     8.021757
2020-01-02     9.264427
2020-01-03    12.575851
2020-01-04    10.387949
2020-01-05    11.840462
Freq: D, Name: y, dtype: float64

,exog_1,exog_2
2020-01-01,8.968465,B
2020-01-02,13.316227,B
2020-01-03,9.405475,A
2020-01-04,7.233246,A
2020-01-05,9.437591,A


In [26]:
# Train RangeDriftDetector
# ==============================================================================
detector = RangeDriftDetector()
detector.fit(series=series, exog=exog)
detector

RangeDriftDetector 
Series value ranges: {'y': (np.float64(5.5850578036003915), np.float64(14.579819894629157))} 
Exogenous value ranges: {'exog_1': (np.float64(4.5430286262543085), np.float64(14.531041199734418)), 'exog_2': {'A', 'B', 'E', 'D', 'C'}} 
Fitted series: ['y'] 
Fitted exogenous: ['exog_1', 'exog_2'] 

Lets assume the model is deployed in production and new data is being used to forecast future values. We simulate a covariate drift in the target series and in the exogenous variables to illustrate how to use the `RangeDriftDetector` class to detect it.

In [27]:
last_window = pd.Series([6.6, 7.5, 100, 9.3, 10.2], name='y') # Value 100 is out of range
last_window_exog = pd.DataFrame({
        'exog_1': [8, 9, 10, 70, 12], # Value 70 is out of range
        'exog_2': ['A', 'B', 'C', 'D', 'W'] # Value 'W' is out of range
    }
)
any_out_of_range, series_out_of_range, exog_out_of_range = detector.predict(
    last_window       = last_window,
    exog              = last_window_exog,
    verbose           = True,
    suppress_warnings = False
)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y' has values outside the range seen during training [5.58506, 14.57982]. This may  │
│ affect the accuracy of the predictions.                                              │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/drift_detection/_range_drift.py:186                                             │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog_1' has values outside the range seen during training [4.54303, 14.53104]. This │
│ may affect the accuracy of the predictions.                                          │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/drift_detection/_range_drift.py:186                                             │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog_2' has values not seen during training. Seen values: {'A', 'B', 'E', 'D',      │
│ 'C'}. This may affect the accuracy of the predictions.                               │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/drift_detection/_range_drift.py:186                                             │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────── Out-of-range summary ──────────────────────────────╮
│ Series:                                                                         │
│ 'y' has values outside the observed range [5.58506, 14.57982].                  │
│                                                                                 │
│ Exogenous Variables:                                                            │
│ 'exog_1' has values outside the observed range [4.54303, 14.53104].             │
│ 'exog_2' has values not seen during training. Seen values: {'A', 'B', 'E', 'D', │
│ 'C'}.                                                                           │
│                                                                                 │
│                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────╯

Same process can be applied when modeling multiple time series. In the case, if exogenous variables are used, the drift detector will check them grouped by series.

In [37]:
# Simulated data - Multiple time series
# ==============================================================================
index = pd.MultiIndex.from_product(
    [['series_1', 'series_2', 'series_3'], pd.date_range(start="2020-01-01", periods=3)],
    names=['series_id', 'datetime']
)
series = pd.DataFrame({
        'y': [1, 2, 3, 10, 20, 30, 100, 200, 300]
    }, index=index
)
exog = pd.DataFrame({
        'exog_1': [5.0, 6.0, 7.0, 15.0, 25.0, 35.0, 150.0, 250.0, 350.0],
        'exog_2': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
    }, index=index
)
display(series)
display(exog)


y
series_id datetime       
series_1  2020-01-01    1
          2020-01-02    2
          2020-01-03    3
series_2  2020-01-01   10
          2020-01-02   20
          2020-01-03   30
series_3  2020-01-01  100
          2020-01-02  200
          2020-01-03  300

exog_1 exog_2
series_id datetime                 
series_1  2020-01-01     5.0      A
          2020-01-02     6.0      B
          2020-01-03     7.0      C
series_2  2020-01-01    15.0      D
          2020-01-02    25.0      E
          2020-01-03    35.0      F
series_3  2020-01-01   150.0      G
          2020-01-02   250.0      H
          2020-01-03   350.0      I

In [38]:
# Train RangeDriftDetector - Multiple time series
# ==============================================================================
detector = RangeDriftDetector()
detector.fit(series=series, exog=exog)
detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.int64(1), np.int64(3)), 'series_2': (np.int64(10), np.int64(30)), 'series_3': (np.int64(100), np.int64(300))} 
Exogenous value ranges: {'series_1': {'exog_1': (np.float64(5.0), np.float64(7.0)), 'exog_2': {'C', 'A', 'B'}}, 'series_2': {'exog_1': (np.float64(15.0), np.float64(35.0)), 'exog_2': {'D', 'F', 'E'}}, 'series_3': {'exog_1': (np.float64(150.0), np.float64(350.0)), 'exog_2': {'H', 'G', 'I'}}} 
Fitted series: ['series_1', 'series_2', 'series_3'] 
Fitted exogenous: ['exog_1', 'exog_2'] 

In [44]:
# New data - Multiple time series
# ==============================================================================
rgn = np.random.default_rng(123)
index = pd.MultiIndex.from_product(
    [['series_1', 'series_2', 'series_3'], pd.date_range(start="2020-01-06", periods=2)],
    names=['series_id', 'datetime']
)
last_window = pd.DataFrame({
        'y': [1.5, 2.3, 100, 20, 110, 200]
    }, index=index
) # Value 100 is out of range
last_window_exog = pd.DataFrame({
        'exog_1': [5.0, 6.1, 10, 70, 220, 290],
        'exog_2': ['A', 'B', 'D', 'F', 'W', 'E'] 
    }, index=index
)
any_out_of_range, series_out_of_range, exog_out_of_range = detector.predict(
    last_window       = last_window,
    exog              = last_window_exog,
    verbose           = True,
    suppress_warnings = False
)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has values outside the range seen during training [10.00000, 30.00000].   │
│ This may affect the accuracy of the predictions.                                     │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/drift_detection/_range_drift.py:186                                             │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'exog_1' has values outside the range seen during training [15.00000,    │
│ 35.00000]. This may affect the accuracy of the predictions.                          │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/drift_detection/_range_drift.py:186                                             │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_3': 'exog_2' has values not seen during training. Seen values: {'H', 'G',    │
│ 'I'}. This may affect the accuracy of the predictions.                               │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/miniconda3/envs/skforecast_16_py12/lib/python3.12/site-packages/skfore │
│ cast/drift_detection/_range_drift.py:186                                             │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Out-of-range summary ──────────────────────────────╮
│ Series:                                                                          │
│ 'series_2' has values outside the observed range [10.00000, 30.00000].           │
│                                                                                  │
│ Exogenous Variables:                                                             │
│ 'series_2': 'exog_1' has values outside the observed range [15.00000, 35.00000]. │
│ 'series_3': 'exog_2' has values not seen during training. Seen values: {'H',     │
│ 'G', 'I'}.                                                                       │
│                                                                                  │
│                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────╯